# 🎯 T5 Fine-Tuning — AU-Ggregates Text-to-SQL (Lightning AI)

Fine-tunes `gaussalgo/T5-LM-Large-text2sql-spider` on your custom + Spider training data.

## Before you start
1. Create a new Studio on [lightning.ai](https://lightning.ai) with **GPU** (L4 recommended, 24GB VRAM)
2. Upload your `t5_text2sql_5000_pairs.jsonl` file to the studio file browser
3. Open this notebook and run all cells

## Pipeline
| Step | Cell | What it does | Time |
|------|------|-------------|------|
| 1 | Install | Install dependencies | ~2 min |
| 2 | GPU Check | Verify GPU | instant |
| 3 | Load Data | Point to uploaded JSONL file | instant |
| 4 | Validate | Validate all 5,000 pairs | ~10 sec |
| 5 | Clean + Merge | Clean custom pairs + merge Spider data | ~3-5 min |
| 6 | Train | Fine-tune T5 (10 epochs) | ~45-90 min |
| 7 | Evaluate | Test accuracy on validation set | ~5 min |
| 8 | Test | Interactive SQL generation test | instant |
| 9 | Export | Zip model for download | ~2 min |

In [ ]:
# ============================================================
# CELL 1: Install dependencies
# ============================================================
!pip install -q torch transformers accelerate sentencepiece
!pip install -q datasets evaluate sqlparse
!pip install -q huggingface_hub

print('\n✅ All dependencies installed!')

In [ ]:
# ============================================================
# CELL 2: Verify GPU
# ============================================================
import torch

print(f'PyTorch: {torch.__version__}')
print(f'CUDA:    {torch.cuda.is_available()}')

if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_mem / 1024**3
    print(f'GPU:     {gpu} ({vram:.1f} GB VRAM)')
    print('✅ GPU ready!')
else:
    print('❌ No GPU! Create a new Studio with GPU enabled.')

In [ ]:
# ============================================================
# CELL 3: Load training data
# ============================================================
# Upload your JSONL file to the Lightning AI file browser first,
# then set the path below.
import os
from pathlib import Path

# --- Set your file path here ---
# Lightning AI default workspace: /teamspace/studios/this_studio/
TRAINING_FILE = '/teamspace/studios/this_studio/t5_text2sql_5000_pairs.jsonl'

# Auto-detect: check common locations
if not os.path.exists(TRAINING_FILE):
    candidates = [
        '/teamspace/studios/this_studio/t5_text2sql_5000_pairs.jsonl',
        '/home/zeus/t5_text2sql_5000_pairs.jsonl',
        './t5_text2sql_5000_pairs.jsonl',
        'data/t5_text2sql_5000_pairs.jsonl',
    ]
    for c in candidates:
        if os.path.exists(c):
            TRAINING_FILE = c
            break

if os.path.exists(TRAINING_FILE):
    size_mb = os.path.getsize(TRAINING_FILE) / 1024 / 1024
    print(f'✅ Found: {TRAINING_FILE} ({size_mb:.1f} MB)')
else:
    print(f'❌ File not found: {TRAINING_FILE}')
    print('   Upload your JSONL file to the file browser and update the path above.')

In [ ]:
# ============================================================
# CELL 4: Validate training data
# ============================================================
import json
import re
from collections import Counter

SCHEMA_PREFIX = 'tables: ai_documents (id, source_table, file_name, project_name, searchable_text, metadata, document_type) | query:'
VALID_SOURCE_TABLES = {'Expenses', 'CashFlow', 'Project', 'Quotation', 'QuotationItem'}
NUMERIC_KEYS = {'Expenses', 'Amount', 'total_amount', 'volume', 'line_total'}

pairs = []
errors = []
source_table_counts = Counter()
intent_counts = Counter()

with open(TRAINING_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue
        try:
            pair = json.loads(line)
        except json.JSONDecodeError as e:
            errors.append(f'Line {i}: Invalid JSON - {e}')
            continue

        inp = pair.get('input', '')
        tgt = pair.get('target', '')

        if 'input' not in pair or 'target' not in pair:
            errors.append(f'Line {i}: Missing input or target key')
            continue

        if not inp.startswith(SCHEMA_PREFIX):
            errors.append(f'Line {i}: Missing Spider schema prefix')

        tgt_upper = tgt.strip().upper()
        if not tgt_upper.startswith('SELECT'):
            errors.append(f'Line {i}: Target is not a SELECT statement')

        has_source = any(f"source_table = '{t}'" in tgt for t in VALID_SOURCE_TABLES)
        if not has_source:
            errors.append(f'Line {i}: Missing source_table filter')

        if "document_type = 'file'" not in tgt and "document_type = 'row'" not in tgt:
            errors.append(f'Line {i}: Missing document_type filter')

        for t in VALID_SOURCE_TABLES:
            if f"source_table = '{t}'" in tgt:
                source_table_counts[t] += 1

        if 'SUM(' in tgt_upper:
            intent_counts['sum'] += 1
        elif 'AVG(' in tgt_upper:
            intent_counts['average'] += 1
        elif 'COUNT(' in tgt_upper:
            intent_counts['count'] += 1
        elif 'GROUP BY' in tgt_upper and ('SUM' in tgt_upper or 'COUNT' in tgt_upper):
            intent_counts['compare'] += 1
        elif 'DISTINCT' in tgt_upper:
            intent_counts['list_categories'] += 1
        elif "document_type = 'file'" in tgt:
            intent_counts['list_files'] += 1
        else:
            intent_counts['query_data'] += 1

        pairs.append(pair)

print('=' * 60)
print(f'📊 VALIDATION REPORT')
print('=' * 60)
print(f'Total pairs:  {len(pairs)}')
print(f'Errors:       {len(errors)}')
print()
print('Source table distribution:')
for t in sorted(source_table_counts, key=source_table_counts.get, reverse=True):
    pct = source_table_counts[t] / len(pairs) * 100
    print(f'  {t:20s} {source_table_counts[t]:5d} ({pct:.1f}%)')
print()
print('Intent distribution:')
for intent in sorted(intent_counts, key=intent_counts.get, reverse=True):
    pct = intent_counts[intent] / len(pairs) * 100
    print(f'  {intent:20s} {intent_counts[intent]:5d} ({pct:.1f}%)')
print()

if errors:
    print(f'⚠️  First 10 errors:')
    for e in errors[:10]:
        print(f'  {e}')
    print()

if len(pairs) >= 4500 and len(errors) < len(pairs) * 0.05:
    print(f'✅ Data looks good! {len(pairs)} valid pairs ready.')
elif len(pairs) > 0:
    print(f'⚠️  {len(pairs)} valid pairs. Error rate: {len(errors)/max(len(pairs),1)*100:.1f}%')
else:
    print('❌ No valid pairs found! Check your JSONL file.')

In [ ]:
# ============================================================
# CELL 5: Clean custom pairs + Merge with Spider dataset
# ============================================================
import json, re, hashlib, random
from collections import Counter
from datasets import load_dataset

SPIDER_LIMIT = 3000

def normalize_sql(sql):
    sql = sql.strip().rstrip(';').strip()
    sql = re.sub(r'\\s+', ' ', sql)
    return sql.lower()

def sql_fingerprint(pair):
    return hashlib.md5(normalize_sql(pair.get('target', '')).encode()).hexdigest()

TYPO_FIXES = {
    "source_table = 'expenses'": "source_table = 'Expenses'",
    "source_table = 'cashflow'": "source_table = 'CashFlow'",
    "source_table = 'Cashflow'": "source_table = 'CashFlow'",
    "source_table = 'cash_flow'": "source_table = 'CashFlow'",
    "source_table = 'project'": "source_table = 'Project'",
    "source_table = 'quotation'": "source_table = 'Quotation'",
    "source_table = 'quotationitem'": "source_table = 'QuotationItem'",
    "source_table = 'QuotationItems'": "source_table = 'QuotationItem'",
    "source_table = 'Quotation_Item'": "source_table = 'QuotationItem'",
}

def clean_pair(pair):
    tgt = pair['target'].strip()
    if not tgt.endswith(';'):
        tgt += ';'
    tgt = tgt.replace(';;', ';')
    for wrong, right in TYPO_FIXES.items():
        tgt = tgt.replace(wrong, right)
    tgt = re.sub(r"metadata->'(\\w+)'", r"metadata->>'\\1'", tgt)
    pair['target'] = tgt
    return pair

def validate_custom(pair):
    inp = pair.get('input', '')
    tgt = pair.get('target', '')
    if not inp.startswith(SCHEMA_PREFIX):
        return False
    if not tgt.strip().upper().startswith('SELECT'):
        return False
    has_src = any(f"source_table = '{t}'" in tgt for t in VALID_SOURCE_TABLES)
    if not has_src:
        return False
    if "document_type = 'file'" not in tgt and "document_type = 'row'" not in tgt:
        return False
    return True

print('🧹 Step 1: Cleaning custom pairs...')
cleaned = [clean_pair(dict(p)) for p in pairs]
valid_custom = [p for p in cleaned if validate_custom(p)]
n_invalid = len(cleaned) - len(valid_custom)
print(f'   Valid: {len(valid_custom)}, Invalid: {n_invalid}')

print(f'\n🕷️ Step 2: Downloading Spider dataset (limit={SPIDER_LIMIT})...')
spider_ds = load_dataset('spider', split='train', trust_remote_code=True)
spider_pairs = []
spider_seen = set()

for ex in spider_ds:
    q = ex.get('question', '')
    sql = ex.get('query', '')
    db = ex.get('db_id', '')
    if not q or not sql:
        continue
    if not sql.strip().upper().startswith('SELECT'):
        continue
    tgt = sql.strip()
    if not tgt.endswith(';'):
        tgt += ';'
    sfp = hashlib.md5(normalize_sql(tgt).encode()).hexdigest()
    if sfp in spider_seen:
        continue
    spider_seen.add(sfp)
    spider_pairs.append({'input': f'tables: {db} | query: {q}', 'target': tgt})
    if len(spider_pairs) >= SPIDER_LIMIT:
        break

print(f'   Spider pairs loaded: {len(spider_pairs)}')

print('\n🔀 Step 3: Merging + deduplicating...')
all_merged = valid_custom + spider_pairs
seen_fps = set()
deduped = []
n_dupes = 0
for p in all_merged:
    fp = sql_fingerprint(p)
    if fp not in seen_fps:
        seen_fps.add(fp)
        deduped.append(p)
    else:
        n_dupes += 1

random.seed(42)
random.shuffle(deduped)

MERGED_FILE = '/teamspace/studios/this_studio/training_final.jsonl'
with open(MERGED_FILE, 'w', encoding='utf-8') as f:
    for p in deduped:
        f.write(json.dumps(p, ensure_ascii=False) + '\n')

TRAINING_FILE = MERGED_FILE

print()
print('=' * 60)
print('  CLEAN + MERGE REPORT')
print('=' * 60)
print(f'  Custom pairs (valid):    {len(valid_custom)}')
print(f'  Custom pairs (invalid):  {n_invalid}')
print(f'  Spider pairs:            {len(spider_pairs)}')
print(f'  Duplicates removed:      {n_dupes}')
print(f'  Total merged:            {len(deduped)}')
print('=' * 60)
if len(deduped) >= 5000:
    print(f'✅ {len(deduped)} pairs ready for training!')
else:
    print(f'⚠️  Only {len(deduped)} pairs.')

In [ ]:
# ============================================================
# CELL 6: Fine-tune T5
# ============================================================
import json
import time
from pathlib import Path
from datasets import Dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

# --- Config ---
MODEL_NAME = 'gaussalgo/T5-LM-Large-text2sql-spider'
OUTPUT_DIR = '/teamspace/studios/this_studio/t5-finetuned'
EPOCHS = 5
BATCH_SIZE = 8          # L4 has 24GB VRAM, can handle batch_size=8
LEARNING_RATE = 3e-5    # low LR for second-stage fine-tuning (prevents catastrophic forgetting)
MAX_INPUT_LEN = 512
MAX_TARGET_LEN = 256

# --- Load data ---
print('📂 Loading training data...')
inputs, targets = [], []
with open(TRAINING_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            pair = json.loads(line)
            if 'input' in pair and 'target' in pair:
                inputs.append(pair['input'])
                targets.append(pair['target'])
        except json.JSONDecodeError:
            continue

dataset = Dataset.from_dict({'input': inputs, 'target': targets})
split = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = split['train']
val_ds = split['test']
print(f'   Train: {len(train_ds)}, Validation: {len(val_ds)}')

# --- Load model ---
print(f'🤖 Loading {MODEL_NAME}...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
print(f'   Parameters: {model.num_parameters():,}')

# --- Tokenize ---
def tokenize(examples):
    model_inputs = tokenizer(
        examples['input'], max_length=MAX_INPUT_LEN,
        truncation=True, padding='max_length'
    )
    labels = tokenizer(
        text_target=examples['target'], max_length=MAX_TARGET_LEN,
        truncation=True, padding='max_length'
    )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

print('🔤 Tokenizing...')
tok_train = train_ds.map(tokenize, batched=True, remove_columns=['input', 'target'])
tok_val = val_ds.map(tokenize, batched=True, remove_columns=['input', 'target'])

# --- Training args ---
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    save_total_limit=2,
    fp16=True,
    report_to='none',
    dataloader_num_workers=2,
    warmup_steps=100,
    weight_decay=0.01,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tok_train,
    eval_dataset=tok_val,
    processing_class=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True),
)

# --- Train ---
print(f'\n🚀 Starting training: {EPOCHS} epochs, batch_size={BATCH_SIZE}, lr={LEARNING_RATE}')
print(f'   Steps per epoch: ~{len(tok_train) // BATCH_SIZE}')
print(f'   Estimated time: ~{EPOCHS * len(tok_train) // BATCH_SIZE * 0.6 / 60:.0f} minutes on L4\n')

start = time.time()
trainer.train()
elapsed = (time.time() - start) / 60

# --- Save ---
save_path = f'{OUTPUT_DIR}/final'
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f'\n✅ Training complete in {elapsed:.1f} minutes!')
print(f'   Model saved to: {save_path}')

In [ ]:
# ============================================================
# CELL 7: Evaluate on validation set
# ============================================================
import sqlparse
import time

print('📊 Evaluating on validation set...\n')

model.eval()
device = model.device

exact_matches = 0
valid_sql = 0
total_time = 0.0
samples = []

for idx in range(len(val_ds)):
    inp = val_ds[idx]['input']
    expected = val_ds[idx]['target']

    encoded = tokenizer(inp, return_tensors='pt', max_length=MAX_INPUT_LEN, truncation=True, padding=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)

    t0 = time.time()
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=MAX_TARGET_LEN)
    total_time += (time.time() - t0) * 1000

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    if generated == expected:
        exact_matches += 1

    try:
        parsed = sqlparse.parse(generated)
        if parsed and parsed[0].get_type() and parsed[0].get_type().upper() == 'SELECT':
            valid_sql += 1
    except Exception:
        pass

    if len(samples) < 15:
        match = '✅' if generated == expected else '❌'
        samples.append((match, inp.split('query: ')[-1], expected, generated))

    if (idx + 1) % 100 == 0:
        print(f'  Evaluated {idx + 1}/{len(val_ds)}...')

total = len(val_ds)
em_acc = exact_matches / total * 100
exec_acc = valid_sql / total * 100
avg_ms = total_time / total

print()
print('=' * 60)
print('  EVALUATION RESULTS')
print('=' * 60)
print(f'  Validation examples:   {total}')
print(f'  Exact-match accuracy:  {em_acc:.1f}%')
print(f'  Valid SQL rate:        {exec_acc:.1f}%')
print(f'  Avg inference time:    {avg_ms:.1f} ms')
print('=' * 60)
print()

if em_acc >= 70:
    print('🎉 Great accuracy! Model is ready for deployment.')
elif em_acc >= 50:
    print('👍 Decent accuracy. Consider more training data or epochs.')
else:
    print('⚠️  Low accuracy. Check training data quality or try more epochs.')

print()
print('Sample predictions:')
print('-' * 60)
for match, question, expected, generated in samples[:10]:
    print(f'{match} Q: {question}')
    print(f'   Expected:  {expected[:100]}')
    print(f'   Generated: {generated[:100]}')
    print()

In [ ]:
# ============================================================
# CELL 8: Interactive test
# ============================================================

SCHEMA_PREFIX_Q = 'tables: ai_documents (id, source_table, file_name, project_name, searchable_text, metadata, document_type) | query: '

def generate_sql(question):
    full_input = SCHEMA_PREFIX_Q + question
    encoded = tokenizer(full_input, return_tensors='pt', max_length=MAX_INPUT_LEN, truncation=True, padding=True)
    input_ids = encoded.input_ids.to(model.device)
    attention_mask = encoded.attention_mask.to(model.device)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=MAX_TARGET_LEN)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

test_questions = [
    'show all expense files',
    'what are the total fuel expenses',
    'how many labor entries are in project alpha',
    'total cash flow amount for highway 5',
    'show approved quotations for manila tower',
    'total volume delivered for plate ABC-1234',
    'list all active projects',
    'average expense amount for materials',
    'how many deliveries used 10-wheeler trucks',
    'compare fuel costs between manila tower and building c',
]

print('🧪 Testing SQL generation:\n')
for q in test_questions:
    sql = generate_sql(q)
    print(f'Q: {q}')
    print(f'→ {sql}')
    print()

In [ ]:
# ============================================================
# CELL 8B: Try your own question (edit and re-run)
# ============================================================

my_question = 'total expenses for SJDM project'  # <-- edit this!

sql = generate_sql(my_question)
print(f'Q: {my_question}')
print(f'→ {sql}')

In [ ]:
# ============================================================
# CELL 9: Export model (zip for download)
# ============================================================
import shutil
from pathlib import Path

FINAL_MODEL = '/teamspace/studios/this_studio/t5-finetuned/final'
ZIP_OUTPUT = '/teamspace/studios/this_studio/t5-finetuned-model'

print('📦 Zipping model for download...')
zip_path = shutil.make_archive(ZIP_OUTPUT, 'zip', FINAL_MODEL)
print(f'   Size: {Path(zip_path).stat().st_size / 1024 / 1024:.0f} MB')
print(f'   File: {zip_path}')
print()
print('✅ Model exported!')
print('   Download the zip from the file browser on the left.')
print('   Extract it and set T5_MODEL_PATH to the extracted folder.')

In [ ]:
# ============================================================
# CELL 9B: Push to HuggingFace Hub (optional)
# ============================================================
# Uncomment and fill in your details to push to HF Hub

# from huggingface_hub import login
# login(token='YOUR_HF_TOKEN')
#
# HF_REPO = 'your-username/t5-auggregates-text2sql'  # <-- change this!
#
# model.push_to_hub(HF_REPO)
# tokenizer.push_to_hub(HF_REPO)
# print(f'✅ Pushed to https://huggingface.co/{HF_REPO}')

---
## 📝 After Training

**To use the fine-tuned model in production:**

1. Download the zip from Cell 9 (use the file browser on the left)
2. Extract to a folder on your server
3. Set `T5_MODEL_PATH` environment variable to the extracted folder path
4. The AI server will automatically use your fine-tuned model

**Expected accuracy targets:**
- Exact-match: 70%+ (good), 80%+ (great)
- Valid SQL rate: 95%+

**If accuracy is low:**
- Check training data quality (run Cell 4 validation)
- Try more epochs (change EPOCHS in Cell 6)
- Try lower learning rate (1e-4 instead of 3e-4)
- Add more diverse training pairs